In [1]:
import pandas as pd

In [2]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

In [3]:
#load data
import pickle
with open('cryptopairs20102020.pkl', 'rb') as handle:
    df = pickle.load(handle)

In [4]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import os

import argparse
import datetime

# The above could be sent to an independent module
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind

#create datafeed class for pandas dataframes
class PandasData(btfeeds.feed.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
    )


class PairTradingStrategy(bt.Strategy):
    params = dict(
        period=10,
        stake=10,
        qty1=0,
        qty2=0,
        printout=True,
        upper=2.1,
        lower=-2.1,
        up_medium=0.5,
        low_medium=-0.5,
        status=0,
        portfolio_value=10000,
        name1='none',
        name2='none'
    )

    def start(self):
        #record results
        if not os.path.exists('mystats.csv'):
            self.mystats = open('mystats.csv', 'w')
            self.mystats.write('date,pair1,pair2,portfolio,period\n')
        else:
            self.mystats = open('mystats.csv', 'a')
            
        if not os.path.exists('endstats.csv'):
            self.mystats2 = open('endstats.csv', 'w')
            self.mystats2.write('pair1,pair2,portfolio,period\n')
        else:
            self.mystats2 = open('endstats.csv', 'a')

        
    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.qty1 = self.p.qty1
        self.qty2 = self.p.qty2
        self.upper_limit = self.p.upper
        self.lower_limit = self.p.lower
        self.up_medium = self.p.up_medium
        self.low_medium = self.p.low_medium
        self.status = self.p.status
        self.portfolio_value = self.p.portfolio_value
        self.pair1 = self.p.name1
        self.pair2 = self.p.name2

        # Signals performed with PD.OLS :
        self.transform = btind.OLS_TransformationN(self.data0, self.data1,
                                                   period=self.p.period)
        self.zscore = self.transform.zscore

        # Checking signals built with StatsModel.API :
        # self.ols_transfo = btind.OLS_Transformation(self.data0, self.data1,
        #                                             period=self.p.period,
        #                                             plot=True)

    def next(self):
        
        self.mystats.write(self.data.datetime.date(0).strftime('%Y-%m-%d'))
        self.mystats.write(',{}'.format(self.pair1))
        
        self.mystats.write(',{}'.format(self.pair2))
        #self.mystats.write(',%.2f' % self.stats.drawdown.drawdown[-1])
        #self.mystats.write(',%.2f' % self.stats.drawdown.maxdrawdown[-1])
        self.mystats.write(',{}'.format(self.broker.getvalue()))
        self.mystats.write(',{}'.format(self.p.period))
        self.mystats.write('\n')

        if self.orderid:
            return  # if an order is active, no new orders are allowed

        '''if self.p.printout:
            print('Self  len:', len(self))
            print('Data0 len:', len(self.data0))
            print('Data1 len:', len(self.data1))
            print('Data0 len == Data1 len:',
                  len(self.data0) == len(self.data1))

            print('Data0 dt:', self.data0.datetime.datetime())
            print('Data1 dt:', self.data1.datetime.datetime())
        '''
        #print('status is', self.status)
        #print('zscore is', self.zscore[0])
        #print('position is', self.position)
        #print('portfolio value:', self.broker.getvalue())

        # Step 2: Check conditions for SHORT & place the order
        # Checking the condition for SHORT
        if (self.zscore[0] > self.upper_limit) and (self.status != 1):

            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Placing the order
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.sell(data=self.data0, size=(x + self.qty1))  # Place an order for buying y + qty2 shares
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.buy(data=self.data1, size=(y + self.qty2))  # Place an order for selling x + qty1 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares

            self.status = 1  # The current status is "short the spread"

            # Step 3: Check conditions for LONG & place the order
            # Checking the condition for LONG
        elif (self.zscore[0] < self.lower_limit) and (self.status != 2):

            # Calculating the number of shares for each stock
            value = 0.5 * self.portfolio_value  # Divide the cash equally
            x = int(value / (self.data0.close))  # Find the number of shares for Stock1
            y = int(value / (self.data1.close))  # Find the number of shares for Stock2
            print('x + self.qty1 is', x + self.qty1)
            print('y + self.qty2 is', y + self.qty2)

            # Place the order
            self.log('BUY CREATE %s, price = %.2f, qty = %d' % (name1, self.data0.close[0], x + self.qty1))
            self.buy(data=self.data0, size=(x + self.qty1))  # Place an order for buying x + qty1 shares
            self.log('SELL CREATE %s, price = %.2f, qty = %d' % (name2, self.data1.close[0], y + self.qty2))
            self.sell(data=self.data1, size=(y + self.qty2))  # Place an order for selling y + qty2 shares

            # Updating the counters with new value
            self.qty1 = x  # The new open position quantity for Stock1 is x shares
            self.qty2 = y  # The new open position quantity for Stock2 is y shares
            self.status = 2  # The current status is "long the spread"


            # Step 4: Check conditions for No Trade
            # If the z-score is within the two bounds, close all
        """
        elif (self.zscore[0] < self.up_medium and self.zscore[0] > self.low_medium):
            self.log('CLOSE LONG %s, price = %.2f' % ("PEP", self.data0.close[0]))
            self.close(self.data0)
            self.log('CLOSE LONG %s, price = %.2f' % ("KO", self.data1.close[0]))
            self.close(self.data1)
        """

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')
        self.mystats2.write('{}'.format(self.pair1))
        
        self.mystats2.write(',{}'.format(self.pair2))
        #self.mystats.write(',%.2f' % self.stats.drawdown.drawdown[-1])
        #self.mystats.write(',%.2f' % self.stats.drawdown.maxdrawdown[-1])
        self.mystats2.write(',{}'.format(self.broker.getvalue()))
        self.mystats2.write(',{}'.format(self.p.period))
        self.mystats2.write('\n')
        self.mystats.close()
        self.mystats2.close()


def runstrategy(name1,name2,period,cash):


    # Create a cerebro
    cerebro = bt.Cerebro()

    # Get the dates from the args


    # Create the 1st data
    data0 = bt.feeds.PandasData(dataname=df[name1])

    # Add the 1st data to cerebro
    cerebro.adddata(data0)

    # Create the 2nd data
    data1 = bt.feeds.PandasData(dataname=df[name2])

    # Add the 2nd data to cerebro
    cerebro.adddata(data1)

    # Add the strategy
    cerebro.addstrategy(PairTradingStrategy,
                        period=period,
                        stake=10,name1=name1,name2=name2)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcash(cash)

    # Add the commission - only stocks like a for each operation
    cerebro.broker.setcommission(commission=0.0)

    # And run it
    cerebro.run()

    # Plot if requested
    #if True:
        #cerebro.plot(numfigs=1, volume=False, zdown=False)
    return cerebro

In [ ]:
# search for results for different periods and pairs
# note order of the pair matters
for period in range(8,30):
    for name1 in df.keys():
        for name2 in df.keys():
            if name1==name2:
                None
            else:
                print(period,name1,name2)
                runstrategy(name1,name2,period,10000)

8 BNBUSDT ETHUSDT
x + self.qty1 is 384
y + self.qty2 is 39
2020-01-02T00:00:00, BUY CREATE BNBUSDT, price = 13.01, qty = 384
2020-01-02T00:00:00, SELL CREATE ETHUSDT, price = 127.19, qty = 39
2020-01-03T00:00:00, BUY COMPLETE, 13.01
2020-01-03T00:00:00, SELL COMPLETE, 127.19
x + self.qty1 is 667
y + self.qty2 is 69
2020-01-15T00:00:00, SELL CREATE BNBUSDT, price = 17.67, qty = 667
2020-01-15T00:00:00, BUY CREATE ETHUSDT, price = 166.40, qty = 69
2020-01-16T00:00:00, SELL COMPLETE, 17.67
2020-01-16T00:00:00, BUY COMPLETE, 166.40
x + self.qty1 is 611
y + self.qty2 is 59
2020-04-06T00:00:00, BUY CREATE BNBUSDT, price = 15.20, qty = 611
2020-04-06T00:00:00, SELL CREATE ETHUSDT, price = 171.32, qty = 59
2020-04-07T00:00:00, BUY COMPLETE, 15.19
2020-04-07T00:00:00, SELL COMPLETE, 171.32
x + self.qty1 is 660
y + self.qty2 is 60
2020-04-13T00:00:00, SELL CREATE BNBUSDT, price = 15.03, qty = 660
2020-04-13T00:00:00, BUY CREATE ETHUSDT, price = 156.29, qty = 60
2020-04-14T00:00:00, SELL COMPLETE

x + self.qty1 is 37
y + self.qty2 is 0
2019-12-16T00:00:00, BUY CREATE ETHUSDT, price = 132.73, qty = 37
2019-12-16T00:00:00, SELL CREATE BTCUSDT, price = 6891.72, qty = 0
2019-12-17T00:00:00, BUY COMPLETE, 132.72
x + self.qty1 is 74
y + self.qty2 is 0
2019-12-29T00:00:00, SELL CREATE ETHUSDT, price = 134.36, qty = 74
2019-12-29T00:00:00, BUY CREATE BTCUSDT, price = 7388.24, qty = 0
2019-12-30T00:00:00, SELL COMPLETE, 134.36
x + self.qty1 is 72
y + self.qty2 is 0
2020-01-07T00:00:00, BUY CREATE ETHUSDT, price = 142.80, qty = 72
2020-01-07T00:00:00, SELL CREATE BTCUSDT, price = 8145.28, qty = 0
2020-01-08T00:00:00, BUY COMPLETE, 142.80
x + self.qty1 is 65
y + self.qty2 is 0
2020-01-14T00:00:00, SELL CREATE ETHUSDT, price = 165.64, qty = 65
2020-01-14T00:00:00, BUY CREATE BTCUSDT, price = 8810.01, qty = 0
2020-01-15T00:00:00, SELL COMPLETE, 165.60
x + self.qty1 is 51
y + self.qty2 is 0
2020-02-09T00:00:00, BUY CREATE ETHUSDT, price = 228.49, qty = 51
2020-02-09T00:00:00, SELL CREATE BTCU

x + self.qty1 is 0
y + self.qty2 is 26
2019-12-18T00:00:00, SELL CREATE BTCUSDT, price = 7277.83, qty = 0
2019-12-18T00:00:00, BUY CREATE BCHUSDT, price = 189.00, qty = 26
2019-12-19T00:00:00, BUY COMPLETE, 188.90
x + self.qty1 is 0
y + self.qty2 is 51
2020-01-02T00:00:00, BUY CREATE BTCUSDT, price = 6965.71, qty = 0
2020-01-02T00:00:00, SELL CREATE BCHUSDT, price = 195.49, qty = 51
2020-01-03T00:00:00, SELL COMPLETE, 195.49
x + self.qty1 is 0
y + self.qty2 is 45
2020-01-07T00:00:00, SELL CREATE BTCUSDT, price = 8145.28, qty = 0
2020-01-07T00:00:00, BUY CREATE BCHUSDT, price = 242.30, qty = 45
2020-01-08T00:00:00, BUY COMPLETE, 242.48
x + self.qty1 is 1
y + self.qty2 is 54
2020-03-12T00:00:00, BUY CREATE BTCUSDT, price = 4800.00, qty = 1
2020-03-12T00:00:00, SELL CREATE BCHUSDT, price = 144.13, qty = 54
2020-03-13T00:00:00, Margin ,
2020-03-13T00:00:00, SELL COMPLETE, 144.13
x + self.qty1 is 1
y + self.qty2 is 55
2020-04-23T00:00:00, SELL CREATE BTCUSDT, price = 7482.39, qty = 1
2020-0

x + self.qty1 is 24
y + self.qty2 is 26311
2019-12-27T00:00:00, SELL CREATE BCHUSDT, price = 203.21, qty = 24
2019-12-27T00:00:00, BUY CREATE XRPUSDT, price = 0.19, qty = 26311
2019-12-28T00:00:00, SELL COMPLETE, 203.33
2019-12-28T00:00:00, BUY COMPLETE, 0.19
x + self.qty1 is 38
y + self.qty2 is 46838
2020-01-18T00:00:00, BUY CREATE BCHUSDT, price = 339.01, qty = 38
2020-01-18T00:00:00, SELL CREATE XRPUSDT, price = 0.24, qty = 46838
2020-01-19T00:00:00, Margin ,
2020-01-19T00:00:00, SELL COMPLETE, 0.24
x + self.qty1 is 27
y + self.qty2 is 42193
2020-01-27T00:00:00, SELL CREATE BCHUSDT, price = 362.86, qty = 27
2020-01-27T00:00:00, BUY CREATE XRPUSDT, price = 0.23, qty = 42193
2020-01-28T00:00:00, SELL COMPLETE, 362.90
2020-01-28T00:00:00, BUY COMPLETE, 0.23
x + self.qty1 is 23
y + self.qty2 is 38098
2020-02-12T00:00:00, BUY CREATE BCHUSDT, price = 473.55, qty = 23
2020-02-12T00:00:00, SELL CREATE XRPUSDT, price = 0.30, qty = 38098
2020-02-13T00:00:00, BUY COMPLETE, 473.55
2020-02-13T00

x + self.qty1 is 22411
y + self.qty2 is 20
2020-01-06T00:00:00, SELL CREATE XRPUSDT, price = 0.22, qty = 22411
2020-01-06T00:00:00, BUY CREATE BCHUSDT, price = 244.25, qty = 20
2020-01-07T00:00:00, SELL COMPLETE, 0.22
2020-01-07T00:00:00, BUY COMPLETE, 244.26
x + self.qty1 is 47764
y + self.qty2 is 41
2020-07-15T00:00:00, BUY CREATE XRPUSDT, price = 0.20, qty = 47764
2020-07-15T00:00:00, SELL CREATE BCHUSDT, price = 227.40, qty = 41
2020-07-16T00:00:00, BUY COMPLETE, 0.20
2020-07-16T00:00:00, SELL COMPLETE, 227.42
x + self.qty1 is 42712
y + self.qty2 is 38
2020-08-02T00:00:00, SELL CREATE XRPUSDT, price = 0.29, qty = 42712
2020-08-02T00:00:00, BUY CREATE BCHUSDT, price = 284.10, qty = 38
2020-08-03T00:00:00, SELL COMPLETE, 0.29
2020-08-03T00:00:00, BUY COMPLETE, 284.10
x + self.qty1 is 38422
y + self.qty2 is 39
2020-09-05T00:00:00, BUY CREATE XRPUSDT, price = 0.24, qty = 38422
2020-09-05T00:00:00, SELL CREATE BCHUSDT, price = 225.71, qty = 39
2020-09-06T00:00:00, BUY COMPLETE, 0.24
202

...

x + self.qty1 is 384
y + self.qty2 is 39
2020-01-02T00:00:00, BUY CREATE BNBUSDT, price = 13.01, qty = 384
2020-01-02T00:00:00, SELL CREATE ETHUSDT, price = 127.19, qty = 39
2020-01-03T00:00:00, BUY COMPLETE, 13.01
2020-01-03T00:00:00, SELL COMPLETE, 127.19
x + self.qty1 is 667
y + self.qty2 is 69
2020-01-15T00:00:00, SELL CREATE BNBUSDT, price = 17.67, qty = 667
2020-01-15T00:00:00, BUY CREATE ETHUSDT, price = 166.40, qty = 69
2020-01-16T00:00:00, SELL COMPLETE, 17.67
2020-01-16T00:00:00, BUY COMPLETE, 166.40
x + self.qty1 is 611
y + self.qty2 is 59
2020-04-06T00:00:00, BUY CREATE BNBUSDT, price = 15.20, qty = 611
2020-04-06T00:00:00, SELL CREATE ETHUSDT, price = 171.32, qty = 59
2020-04-07T00:00:00, BUY COMPLETE, 15.19
2020-04-07T00:00:00, SELL COMPLETE, 171.32
x + self.qty1 is 660
y + self.qty2 is 60
2020-04-13T00:00:00, SELL CREATE BNBUSDT, price = 15.03, qty = 660
2020-04-13T00:00:00, BUY CREATE ETHUSDT, price = 156.29, qty = 60
2020-04-14T00:00:00, SELL COMPLETE, 15.03
2020-04-14

x + self.qty1 is 30
y + self.qty2 is 0
2020-01-14T00:00:00, SELL CREATE ETHUSDT, price = 165.64, qty = 30
2020-01-14T00:00:00, BUY CREATE BTCUSDT, price = 8810.01, qty = 0
2020-01-15T00:00:00, SELL COMPLETE, 165.60
x + self.qty1 is 51
y + self.qty2 is 0
2020-03-05T00:00:00, BUY CREATE ETHUSDT, price = 228.38, qty = 51
2020-03-05T00:00:00, SELL CREATE BTCUSDT, price = 9054.68, qty = 0
2020-03-06T00:00:00, BUY COMPLETE, 228.38
x + self.qty1 is 50
y + self.qty2 is 0
2020-04-06T00:00:00, SELL CREATE ETHUSDT, price = 171.32, qty = 50
2020-04-06T00:00:00, BUY CREATE BTCUSDT, price = 7329.90, qty = 0
2020-04-07T00:00:00, SELL COMPLETE, 171.32
x + self.qty1 is 55
y + self.qty2 is 0
2020-04-23T00:00:00, BUY CREATE ETHUSDT, price = 185.51, qty = 55
2020-04-23T00:00:00, SELL CREATE BTCUSDT, price = 7482.39, qty = 0
2020-04-24T00:00:00, BUY COMPLETE, 185.57
x + self.qty1 is 46
y + self.qty2 is 0
2020-05-30T00:00:00, SELL CREATE ETHUSDT, price = 243.66, qty = 46
2020-05-30T00:00:00, BUY CREATE BTCU

x + self.qty1 is 0
y + self.qty2 is 20
2020-01-07T00:00:00, SELL CREATE BTCUSDT, price = 8145.28, qty = 0
2020-01-07T00:00:00, BUY CREATE BCHUSDT, price = 242.30, qty = 20
2020-01-08T00:00:00, BUY COMPLETE, 242.48
x + self.qty1 is 1
y + self.qty2 is 54
2020-03-12T00:00:00, BUY CREATE BTCUSDT, price = 4800.00, qty = 1
2020-03-12T00:00:00, SELL CREATE BCHUSDT, price = 144.13, qty = 54
2020-03-13T00:00:00, BUY COMPLETE, 4800.01
2020-03-13T00:00:00, SELL COMPLETE, 144.13
x + self.qty1 is 1
y + self.qty2 is 56
2020-03-23T00:00:00, SELL CREATE BTCUSDT, price = 6467.31, qty = 1
2020-03-23T00:00:00, BUY CREATE BCHUSDT, price = 221.26, qty = 56
2020-03-24T00:00:00, SELL COMPLETE, 6465.25
2020-03-24T00:00:00, BUY COMPLETE, 221.23
x + self.qty1 is 0
y + self.qty2 is 41
2020-05-02T00:00:00, BUY CREATE BTCUSDT, price = 8972.05, qty = 0
2020-05-02T00:00:00, SELL CREATE BCHUSDT, price = 260.84, qty = 41
2020-05-03T00:00:00, SELL COMPLETE, 260.84
x + self.qty1 is 0
y + self.qty2 is 39
2020-05-04T00:00

x + self.qty1 is 24
y + self.qty2 is 0
2019-12-27T00:00:00, SELL CREATE BCHUSDT, price = 203.21, qty = 24
2019-12-27T00:00:00, BUY CREATE BTCUSDT, price = 7254.74, qty = 0
2019-12-28T00:00:00, SELL COMPLETE, 203.33
x + self.qty1 is 44
y + self.qty2 is 0
2020-05-06T00:00:00, BUY CREATE BCHUSDT, price = 241.32, qty = 44
2020-05-06T00:00:00, SELL CREATE BTCUSDT, price = 9142.92, qty = 0
2020-05-07T00:00:00, BUY COMPLETE, 241.32
x + self.qty1 is 39
y + self.qty2 is 0
2020-05-08T00:00:00, SELL CREATE BCHUSDT, price = 260.30, qty = 39
2020-05-08T00:00:00, BUY CREATE BTCUSDT, price = 9800.01, qty = 0
2020-05-09T00:00:00, SELL COMPLETE, 260.16
x + self.qty1 is 42
y + self.qty2 is 0
2020-06-27T00:00:00, BUY CREATE BCHUSDT, price = 216.37, qty = 42
2020-06-27T00:00:00, SELL CREATE BTCUSDT, price = 9012.00, qty = 0
2020-06-28T00:00:00, BUY COMPLETE, 216.34
x + self.qty1 is 43
y + self.qty2 is 0
2020-07-06T00:00:00, SELL CREATE BCHUSDT, price = 242.34, qty = 43
2020-07-06T00:00:00, BUY CREATE BTCU

x + self.qty1 is 22411
y + self.qty2 is 20
2020-01-06T00:00:00, SELL CREATE XRPUSDT, price = 0.22, qty = 22411
2020-01-06T00:00:00, BUY CREATE BCHUSDT, price = 244.25, qty = 20
2020-01-07T00:00:00, SELL COMPLETE, 0.22
2020-01-07T00:00:00, BUY COMPLETE, 244.26
x + self.qty1 is 47715
y + self.qty2 is 40
2020-05-29T00:00:00, BUY CREATE XRPUSDT, price = 0.20, qty = 47715
2020-05-29T00:00:00, SELL CREATE BCHUSDT, price = 238.43, qty = 40
2020-05-30T00:00:00, BUY COMPLETE, 0.20
2020-05-30T00:00:00, SELL COMPLETE, 238.35
x + self.qty1 is 49633
y + self.qty2 is 40
2020-07-08T00:00:00, SELL CREATE XRPUSDT, price = 0.21, qty = 49633
2020-07-08T00:00:00, BUY CREATE BCHUSDT, price = 243.31, qty = 40
2020-07-09T00:00:00, SELL COMPLETE, 0.21
2020-07-09T00:00:00, BUY COMPLETE, 243.29
x + self.qty1 is 45392
y + self.qty2 is 42
2020-09-05T00:00:00, BUY CREATE XRPUSDT, price = 0.24, qty = 45392
2020-09-05T00:00:00, SELL CREATE BCHUSDT, price = 225.71, qty = 42
2020-09-06T00:00:00, Margin ,
2020-09-06T00

In [ ]:
# daily results saved in mystats.csv
pd.read_csv('mystats.csv')

In [ ]:
#final results saved in endstats.csv
pd.read_csv('endstats.csv')